# Params #  
X -> m-by-n matrix of net returns (samples-by-assets)  
r -> m-dimensional vec of net returns of index  
reg -> sparsity regularization parameter  
u -> upper bound of weights  
w_0 -> initial point  
p_neg_exp -> final negative exponent of p  
max_iter -> max number of iterations  

return n-dimensional vector w/ allocation weights on the assets  

In [ ]:
# TODO: calculateReturns() -> transform close data into return matrix acording to param
# TODO: Add comments to optimization algorithm along with mathematical formulation (Henry)
# TODO: modularize pipeline
# TODO: Look into HUBER tracking error algo (potentially many others to consider although might be a good idea to survey those with most potential)

In [ ]:
import torch
import math
from sec import stock, constants, lookups
from polygon import RESTClient

In [ ]:
constants.set_polygon_key("_4BtZn3PRCLu6fsdu7dgddb4ucmB1sfp")
poly_cli = RESTClient(api_key="_4BtZn3PRCLu6fsdu7dgddb4ucmB1sfp")

In [ ]:
curr_sp = lookups.get_sp500_tickers()
print(len(curr_sp))

In [ ]:
print(curr_sp)

In [ ]:
# aggregate all data
close_data = torch.zeros((252, len(curr_sp)))

for i in range(len(curr_sp)):
    aggs = []
    data = poly_cli.list_aggs(ticker=curr_sp[i], multiplier=1, timespan="day", from_="2023-02-01", to = "2024-02-01")
    for j, agg in enumerate(data):
        close_data[j, i] = agg.close

print(close_data)

In [ ]:
def getTickerClose(ticker, curr_sp, close_data):
    ix = curr_sp.index(ticker)
    print(close_data[:, ix])

getTickerClose("AAPL", curr_sp, close_data)

In [ ]:
def bisection(c, u):
  n = len(c)
  w = torch.zeros(n)

  c_sort, sort_ind = torch.sort(c)

  high = n
  low = 1

  while low <= high:
    mid = (low + high) // 2
    mu = -1/mid * (torch.sum(c_sort[0:mid]) + 2) if mid != 0 else float('-inf')

    cond1 = (mu + c_sort[mid]) < 0
    if mid < n:
      cond2 = (mu + c_sort[mid + 1] >= 0)
    else:
      cond2 = True

    if cond1 and cond2:
      break
    elif cond1 and not cond2:
      low = mid + 1
    else:
      high = mid - 1

  new_values = -(mu + c_sort[:mid]) / 2
  if torch.max(new_values) <= u:
    w[sort_ind[:mid]] = new_values
    return w
  else:
    flag = False
    flag2 = False
    k = mid

    while True:
      low1 = 0
      high1 = k - 1

      while low1 <= high1:
        mid1 = (low1 + high1) // 2
        if mid1 < k - 1:
          mu = (2 * mid1 * u - torch.sum(c_sort[mid1:k]) - 2) / (k - mid1)
        else:
          mu = float('-inf')

        if mid1 != 0:
          cond1 = mu + c_sort[mid1] <= -2*u
        else:
          cond2 = True

        cond2 = ((-2 * u) < (mu + c_sort[mid1 + 1])) and ((mu + c_sort[k]) < 0)

        if k < n:
          cond3 = (mu + c_sort[k+1]) >= 0
        else:
          cond3 = True

        if cond1 and cond2 and cond3:
          flag = True
          break
        elif cond1 and not cond2:
          low1 = mid1 + 1
        else:
          high1 = mid1 - 1

      if flag:
        break
      else:
        k = k + 1

      if k > n:
        flag2 = True
        break

      if flag2:
        num_elements = int(torch.ceil(torch.tensor(1/u).item()))
        w[sort_ind[num_elements]] = u
      else:
        w[sort_ind[:mid1]] = u
        w[sort_ind[mid1:k]] = -(mu + c_sort[mid1:k]) / 2

In [ ]:
def eteMMupdate(w, B, b, Lmax_A, reg, p , c_1, u):
  d = reg / ((p + abs(w)) * c_1)
  c = B @ w + 1/Lmax_A * (b + d)

  return bisection(c,u)

In [ ]:
def track(self, data, returns, reg, u = 1, w_0 = None, p_neg_exp = 7, max_iter = 1000):
  X = data
  n = X.shape[0]
  m = X.shape[1]

  if n == 1:
    print("Must track more than one asset")

  if not w_0: # w_0 = None -> Normal Distribution
    w_0 = torch.ones(n) / n

  F_v = torch.zeros((max_iter, 1))

  K = 10
  p_1 = 1
  p_k = p_neg_exp
  gamma = (p_k/p_1)**(1/K)
  seq = torch.arange(0, K + 1)
  exp = gamma**seq
  p_p = p_1 * exp
  p_p = 10**(-p_p)

  p_p_div_10 = p_p/10
  ones = 1e-3 * torch.ones_like(p_p)
  tol = torch.min(p_p_div_10, ones)

  k = 0 # iter tracker

  # Using Empirical Tracking Error

  A = 1/m * torch.transpose(X) @ X

  eigenvalues, _ = torch.linalg.eigh(A)
  Lmax_A = eigenvalues[-1]

  B = 2/Lmax_A * (A - Lmax_A * torch.eye(n))
  b = -2/m * torch.transpose(X) @ returns

  for i in range(1,K+1):
    p = p_p[i]
    c_1 = torch.log(1 + u/p)
    flag = True

    while True:
      k = k = 1

      w_1 = eteMMupdate(w_0, B, b, Lmax_A, reg, p, c_1, u)
      w_2 = eteMMupdate(w_1, B, b, Lmax_A, reg, p, c_1, u)
      R = w_1 - w_0
      U = w_2 - w_1 - R
      norm_R = torch.norm(R)
      norm_U = torch.norm(U)

      a = max(min(-norm_R / norm_U, -1), -300)

      while True:
        if torch.abs(a + 1) < 1e-6:
          w = w_2
          F_v[k] = 1/reg * torch.norm(torch.matmul(X, w) - returns)**2 + m/c_1 * torch.sum(torch.log(1 + w/p))
          w_0 = w
          break

        w = w_0 - 2 * a * R + a**2 * U

        w = bisection(-2 * w, u)
        F_v[k] = 1/reg * torch.norm(torch.matmul(X, w) - returns)**2 + m/c_1 * torch.sum(torch.log(1 + w/p))

        if flag == 0 and F_v[k] * (1 - torch.sign(F_v[k]) * 1e-9) >= F_v[max(k - 1, 0)]:
          a = (a - 1) / 2
        else:
          w_0 = w
          break

      if flag == 0:
        rel_change = torch.abs(F_v[k] - F_v[k - 1]) / max(1, torch.abs(F_v[k - 1]))

        if rel_change <= tol[i] or k >= max_iter:
          break
      flag = 0
